In [1]:
%cd /home/jkliu/Projects/CSAServer

/home/jkliu/Projects/CSAServer


/home/jkliu/Projects/CSAServer/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from fastapi.testclient import TestClient
from main import app

client = TestClient(app)

cg_container: INFO     Load cg for com.amaze.filemanager-117
cg_container: INFO     Load cg for org.woheller69.spritpreise-24
cg_container: INFO     Load cg for org.wikipedia.alpha-50476
cg_container: INFO     Load cg for org.wikipedia.dev-50496
cg_container: INFO     Load cg for com.zhiliaoapp.musically-2022903010
cg_container: INFO     Load cg for org.wikipedia.dev-50496_bac
cg_container: INFO     Load cg for org.wikipedia.dev-50480
cg_container: INFO     Load cg for org.wikipedia.dev-50443
cg_container: INFO     Load cg for org.woheller69.spritpreise-18
Main        : INFO     Start logging


In [3]:
# test get_acitivities
request = client.build_request(
    method="post",
    url="get_permission?package_name=org.wikipedia.dev&version_code=50496"
)
response = client.send(request)
result = response.json()
print(result["permissions"])
print(result["last_debloated_permissions"])


httpx       : INFO     HTTP Request: POST http://testserver/get_permission?package_name=org.wikipedia.dev&version_code=50496 "HTTP/1.1 200 OK"


['android.permission.WAKE_LOCK', 'android.permission.ACCESS_NETWORK_STATE', 'android.permission.WRITE_EXTERNAL_STORAGE', 'android.permission.ACCESS_COARSE_LOCATION', 'android.permission.POST_NOTIFICATIONS', 'android.permission.FOREGROUND_SERVICE', 'android.permission.READ_EXTERNAL_STORAGE', 'android.permission.ACCESS_FINE_LOCATION']
['android.permission.ACCESS_FINE_LOCATION', 'android.permission.ACCESS_COARSE_LOCATION']


In [4]:
import os


def sign_align_new_apk(input_apk):
    aligned_apk = input_apk.replace(".apk", "_aligned_tmp.apk")
    output_apk = input_apk.replace(".apk", "_signed.apk")
    temp_file = input_apk.replace(".apk", "_signed.apk.idsig")
    os.system(f"zipalign -f 4 {input_apk} {aligned_apk}")
    os.system(f"apksigner sign --ks Apksigner --ks-pass pass:60948491 --out {output_apk} {aligned_apk}")
    # os.system(f"zip -d {output_apk} META-INF/CERT.SF META-INF/CERT.RSA")
    # os.system(f"mv {output_apk} {input_apk}")
    # os.system(f"rm {aligned_apk}")
    # os.system(f"rm {temp_file}")

In [6]:
# test get_acitivities
app_name = "org.wikipedia.dev"
version_code = "50496"
result_dir = "tests/results/temp_debloated"

if os.path.exists(result_dir):
    os.system(f"rm -r {result_dir}")
os.makedirs(result_dir)

from datetime import datetime
import os
request = client.build_request(
    method="post",
    url=f"debloat_permission?package_name={app_name}&version_code={version_code}&permissions=android.permission.ACCESS_FINE_LOCATION,android.permission.ACCESS_COARSE_LOCATION"
)
response = client.send(request)
result = response.json()
to_remove_methods = result["to_remove_methods"]
print(to_remove_methods)
print(result["to_remove_permissions"])

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
current_time = current_time.replace(" ", "_")
with open(f"{result_dir}/{current_time}_to_remove.log", "w") as write2filelog:
    for method in to_remove_methods:
        write2filelog.write(f"{method}\n")

cmd = f"java -jar lib/debloater/target/debloater-1.0-SNAPSHOT-jar-with-dependencies.jar -toDebloatPath {result_dir}/{current_time}_to_remove.log -androidJar lib/platforms -apkPath apks/{app_name}-{version_code}.apk"
os.system(cmd)
sign_align_new_apk(f"{result_dir}/{app_name}-{version_code}.apk")


Main        : INFO     Debloat permission android.permission.ACCESS_FINE_LOCATION,android.permission.ACCESS_COARSE_LOCATION for org.wikipedia.dev-50496
MethodFinder: INFO     Debloat permission ['android.permission.ACCESS_FINE_LOCATION', 'android.permission.ACCESS_COARSE_LOCATION'] for org.wikipedia.dev-50496
httpx       : INFO     HTTP Request: POST http://testserver/debloat_permission?package_name=org.wikipedia.dev&version_code=50496&permissions=android.permission.ACCESS_FINE_LOCATION,android.permission.ACCESS_COARSE_LOCATION "HTTP/1.1 200 OK"


['<org.maplibre.android.location.permissions.PermissionsManager: boolean isFineLocationPermissionGranted(android.content.Context)>', '<org.wikipedia.places.PlacesFragment: boolean haveLocationPermissions()>', '<org.maplibre.android.location.permissions.PermissionsManager: boolean isCoarseLocationPermissionGranted(android.content.Context)>', '<org.wikipedia.places.PlacesFragment: void locationPermissionRequest$lambda$2(org.wikipedia.places.PlacesFragment,java.util.Map)>', '<org.wikipedia.places.PlacesFragment: void onCreateView$lambda$13(org.wikipedia.places.PlacesFragment,android.view.View)>', '<androidx.appcompat.app.TwilightManager: android.location.Location getLastKnownLocation()>', '<org.maplibre.android.location.permissions.PermissionsManager: void requestLocationPermissions(android.app.Activity,boolean,boolean)>', '<org.wikipedia.places.PlacesFragment: void onViewCreated$lambda$25$lambda$24(org.wikipedia.places.PlacesFragment,org.maplibre.android.maps.MapLibreMap,org.maplibre.and

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


Initialized soot
Number of to debloate methods: 9
/home/jkliu/Projects/CSAServer/tests/results/temp_debloatedapks/org.wikipedia.dev-50496.apk
Removed [<org.wikipedia.places.PlacesFragment: void locationPermissionRequest$lambda$2(org.wikipedia.places.PlacesFragment,java.util.Map)>, <org.maplibre.android.location.permissions.PermissionsManager: void requestLocationPermissions(android.app.Activity,boolean,boolean)>, <org.wikipedia.places.PlacesFragment: void onViewCreated$lambda$25$lambda$24(org.wikipedia.places.PlacesFragment,org.maplibre.android.maps.MapLibreMap,org.maplibre.android.maps.Style)>, <org.maplibre.android.location.permissions.PermissionsManager: boolean isFineLocationPermissionGranted(android.content.Context)>, <org.maplibre.android.location.permissions.PermissionsManager: boolean isCoarseLocationPermissionGranted(android.content.Context)>, <androidx.appcompat.app.TwilightManager: android.location.Location getLastKnownLocation()>, <org.wikipedia.places.PlacesFragment: void 

In [7]:
# test get_acitivities
app_name = "org.wikipedia.dev"
version_code = "50496"
result_dir = "tests/results/temp_debloated"

if os.path.exists(result_dir):
    os.system(f"rm -r {result_dir}")
os.makedirs(result_dir)

from datetime import datetime
import os
request = client.build_request(
    method="post",
    url=f"debloat_permission_old?package_name={app_name}&version_code={version_code}&permissions=android.permission.ACCESS_FINE_LOCATION,android.permission.ACCESS_COARSE_LOCATION"
)
response = client.send(request)
result = response.json()
to_remove_methods = result["to_remove_methods"]
print(to_remove_methods)
print(result["to_remove_permissions"])

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
current_time = current_time.replace(" ", "_")
with open(f"{result_dir}/{current_time}_to_remove.log", "w") as write2filelog:
    for method in to_remove_methods:
        write2filelog.write(f"{method}\n")

cmd = f"java -jar lib/debloater/target/debloater-1.0-SNAPSHOT-jar-with-dependencies.jar -toDebloatPath {result_dir}/{current_time}_to_remove.log -androidJar lib/platforms -apkPath apks/{app_name}-{version_code}.apk"
os.system(cmd)
print("*******************FINISH********************")
sign_align_new_apk(f"{result_dir}/{app_name}-{version_code}.apk")


Main        : INFO     Debloat permission android.permission.ACCESS_FINE_LOCATION,android.permission.ACCESS_COARSE_LOCATION for org.wikipedia.dev-50496
MethodFinder: INFO     Debloat permission ['android.permission.ACCESS_FINE_LOCATION', 'android.permission.ACCESS_COARSE_LOCATION'] for org.wikipedia.dev-50496
MethodFinder: INFO     Debloat permission android.permission.ACCESS_FINE_LOCATION for org.wikipedia.dev-50496
MethodFinder: INFO     permission_methods: ['<android.location.LocationManager: android.location.LocationProvider getProvider(java.lang.String)>', '<android.location.LocationManager: void requestLocationUpdates(android.location.LocationRequest,android.app.PendingIntent)>', '<android.location.LocationManager: void requestSingleUpdate(java.lang.String,android.location.LocationListener,android.os.Looper)>', '<android.net.wifi.WifiManager: void addSuggestionConnectionStatusListener(java.util.concurrent.Executor,android.net.wifi.WifiManager$SuggestionConnectionStatusListener)>'

['<android.location.LocationManager: android.location.LocationProvider getProvider(java.lang.String)>', '<android.location.LocationManager: void requestLocationUpdates(android.location.LocationRequest,android.app.PendingIntent)>', '<android.location.LocationManager: void requestSingleUpdate(java.lang.String,android.location.LocationListener,android.os.Looper)>', '<android.net.wifi.WifiManager: void addSuggestionConnectionStatusListener(java.util.concurrent.Executor,android.net.wifi.WifiManager$SuggestionConnectionStatusListener)>', '<android.location.LocationManager: boolean registerGnssNavigationMessageCallback(java.util.concurrent.Executor,android.location.GnssNavigationMessage$Callback)>', '<android.location.LocationManager: void injectGnssMeasurementCorrections(android.location.GnssMeasurementCorrections)>', '<android.hardware.location.GeofenceHardware: boolean addGeofence(int,int,android.hardware.location.GeofenceHardwareRequest,android.hardware.location.GeofenceHardwareCallback)>

APK path: apks/org.wikipedia.dev-50496.apk


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


Initialized soot
Number of to debloate methods: 220
*******************FINISH********************


Exception in thread "Thread-55" Exception in thread "Thread-32" java.util.ConcurrentModificationException
	at java.base/java.util.HashMap$HashIterator.nextNode(HashMap.java:1597)
	at java.base/java.util.HashMap$KeyIterator.next(HashMap.java:1620)
	at com.smu.jkliu.instrument.MethodTransformer.internalTransform(MethodTransformer.java:35)
	at soot.BodyTransformer.transform(BodyTransformer.java:47)
	at soot.Transform.apply(Transform.java:126)
	at soot.BodyPack.internalApply(BodyPack.java:49)
	at soot.Pack.apply(Pack.java:126)
	at soot.PackManager.runBodyPacks(PackManager.java:991)
	at soot.PackManager.lambda$runBodyPacks$0(PackManager.java:667)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1136)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:635)
	at java.base/java.lang.Thread.run(Thread.java:842)
java.util.ConcurrentModificationException
	at java.base/java.util.HashMap$HashIterator.nextNode(HashMap.java:1